# Exponential Smoothing Methods - Solution

This notebook provides a complete solution for implementing exponential smoothing methods at FreshMart, including:
1. Simple Exponential Smoothing (SES)
2. Double Exponential Smoothing (Holt's Method)
3. Parameter Optimization
4. Forecast Accuracy Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from utils.testing.forecasting_tests import check_exponential_smoothing, check_forecast_accuracy

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('husl')

## Part 1: Simple Exponential Smoothing

First, we'll implement simple exponential smoothing with different smoothing parameters (α).

In [ ]:
def simple_exponential_smoothing(data, alpha):
    """Implement simple exponential smoothing.
    
    Args:
        data (array): Input time series data
        alpha (float): Smoothing parameter between 0 and 1
        
    Returns:
        array: Smoothed values
    """
    result = np.zeros_like(data)
    result[0] = data[0]  # Initialize with first value
    
    for t in range(1, len(data)):
        result[t] = alpha * data[t] + (1 - alpha) * result[t-1]
        
    return result

# Sample data: Daily yogurt sales
sales_data = np.array([120, 132, 125, 138, 128, 135, 140, 133, 129, 142,
                       136, 131, 144, 138, 135, 142, 140, 138, 145, 140])

# Calculate smoothed values with different α
smoothed_0_2 = simple_exponential_smoothing(sales_data, 0.2)
smoothed_0_5 = simple_exponential_smoothing(sales_data, 0.5)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(sales_data, marker='o', label='Actual Data')
plt.plot(smoothed_0_2, marker='s', label='α = 0.2')
plt.plot(smoothed_0_5, marker='^', label='α = 0.5')
plt.title('Simple Exponential Smoothing - Yogurt Sales')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate forecast accuracy
mse_0_2 = np.mean((sales_data - smoothed_0_2)**2)
mse_0_5 = np.mean((sales_data - smoothed_0_5)**2)

print("Forecast Accuracy:")
print(f"MSE (α = 0.2): {mse_0_2:.2f}")
print(f"MSE (α = 0.5): {mse_0_5:.2f}")

# Test smoothing implementation
if check_exponential_smoothing(sales_data, smoothed_0_2, 0.2):
    print("✓ Simple exponential smoothing correctly implemented!")
else:
    print("✗ Check your simple exponential smoothing implementation")

### Analysis of α Selection

1. **Effect of α**:
   - Lower α (0.2): More smoothing, slower response to changes
   - Higher α (0.5): Less smoothing, faster response to changes

2. **Trade-offs**:
   - Low α: Better for stable data with noise
   - High α: Better for data with genuine changes

3. **Optimal α Selection**:
   - Use cross-validation
   - Minimize forecast error (MSE)

## Part 2: Double Exponential Smoothing (Holt's Method)

Now we'll implement Holt's method for data with trend.

In [ ]:
def holts_method(data, alpha, beta):
    """Implement Holt's double exponential smoothing.
    
    Args:
        data (array): Input time series data
        alpha (float): Level smoothing parameter
        beta (float): Trend smoothing parameter
        
    Returns:
        tuple: (Smoothed values, Level component, Trend component)
    """
    n = len(data)
    result = np.zeros(n)
    level = np.zeros(n)
    trend = np.zeros(n)
    
    # Initialize
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    result[0] = level[0]
    
    # Calculate level, trend and smoothed values
    for t in range(1, n):
        # Previous smoothed value
        prev_smooth = level[t-1] + trend[t-1]
        
        # Update level and trend
        level[t] = alpha * data[t] + (1 - alpha) * prev_smooth
        trend[t] = beta * (level[t] - level[t-1]) + (1 - beta) * trend[t-1]
        
        # Calculate smoothed value
        result[t] = level[t] + trend[t]
    
    return result, level, trend

# Data with trend
trend_data = np.array([100, 108, 115, 125, 133, 142, 150, 160, 168, 177,
                       185, 195, 203, 212, 220, 230, 238, 247, 255, 265])

# Apply Holt's method
alpha, beta = 0.5, 0.3
holts_forecast, level, trend = holts_method(trend_data, alpha, beta)

# Generate forecasts for next 5 periods
forecast_horizon = 5
forecasts = np.zeros(forecast_horizon)
for h in range(forecast_horizon):
    forecasts[h] = level[-1] + (h + 1) * trend[-1]

# Plot results
plt.figure(figsize=(12, 6))
t = np.arange(len(trend_data))
t_forecast = np.arange(len(trend_data), len(trend_data) + forecast_horizon)

plt.plot(t, trend_data, marker='o', label='Actual Data')
plt.plot(t, holts_forecast, marker='s', label="Holt's Method")
plt.plot(t, simple_exponential_smoothing(trend_data, 0.5), '--', label='Simple Exp. Smoothing')
plt.plot(t_forecast, forecasts, marker='^', label='Forecast')

plt.title("Holt's Method vs Simple Exponential Smoothing")
plt.xlabel('Time Period')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate forecast accuracy
mse_holts = np.mean((trend_data - holts_forecast)**2)
mse_ses = np.mean((trend_data - simple_exponential_smoothing(trend_data, 0.5))**2)

print("Forecast Accuracy Comparison:")
print(f"MSE (Holt's Method): {mse_holts:.2f}")
print(f"MSE (Simple Exp. Smoothing): {mse_ses:.2f}")

# Test forecast accuracy
if check_forecast_accuracy({'holts': mse_holts, 'ses': mse_ses}):
    print("✓ Forecast accuracy correctly calculated!")
else:
    print("✗ Check your forecast accuracy calculations")

### Analysis of Results

1. **When to Use Holt's Method**:
   - Data shows clear trend
   - Trend is relatively consistent
   - Need to forecast multiple periods ahead

2. **Parameter Selection (α and β)**:
   - α controls level smoothing (0.5 gives balanced response)
   - β controls trend smoothing (0.3 prevents over-reaction to trend changes)
   - Can be optimized using grid search or optimization algorithms

3. **Comparison with Simple Exponential Smoothing**:
   - Holt's method performs better with trended data
   - Simple exponential smoothing lags behind in upward trends
   - Holt's method provides more accurate multi-step forecasts